# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

# Must enable gmaps extension to display interactive gmaps in jupyter notebook
# jupyter nbextension enable --py gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load cities.csv from part I
filepath = os.path.join('..', 'WeatherPy', 'Data', 'cities.csv')
cities_data = pd.read_csv(filepath)

# Display DataFrame
cities_data.head()

,City Name,Country,Latitude,Longitude,Max Temperature (*F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Datetime
0,Longyearbyen,SJ,78.2186,15.6401,14.00,79,75,12.66,2021-02-10 10:59:07
1,Husavik,IS,66.0449,-17.3389,23.00,93,90,3.44,2021-02-10 10:59:07
2,Jamestown,US,42.0970,-79.2353,15.01,92,90,5.75,2021-02-10 10:54:14
3,Coari,BR,-4.0850,-63.1414,72.95,97,95,2.42,2021-02-10 10:59:07
4,Florø,NO,61.5996,5.0328,28.40,64,20,1.14,2021-02-10 10:59:08


In [3]:
# Count data points
print(cities_data.count())

# Lat and long already stored as float
cities_data.dtypes

City Name               601
Country                 594
Latitude                601
Longitude               601
Max Temperature (*F)    601
Humidity (%)            601
Cloudiness (%)          601
Wind Speed (MPH)        601
Datetime                601
dtype: int64


City Name                object
Country                  object
Latitude                float64
Longitude               float64
Max Temperature (*F)    float64
Humidity (%)              int64
Cloudiness (%)            int64
Wind Speed (MPH)        float64
Datetime                 object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude data
locations = cities_data[['Latitude', 'Longitude']]
# locations

# Store humidity data
humidity = cities_data['Humidity (%)'].astype(float)
# humidity

# Find max humidity for gmaps max intensity
max_intensity = humidity.max()
# max_intensity

In [6]:
# Create map
fig = gmaps.figure()

# Add heatmap layer displaying humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity=max_intensity, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Drop rows with null values
no_null_cities = cities_data.dropna()
no_null_cities

,City Name,Country,Latitude,Longitude,Max Temperature (*F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Datetime
0,Longyearbyen,SJ,78.2186,15.6401,14.00,79,75,12.66,2021-02-10 10:59:07
1,Husavik,IS,66.0449,-17.3389,23.00,93,90,3.44,2021-02-10 10:59:07
2,Jamestown,US,42.0970,-79.2353,15.01,92,90,5.75,2021-02-10 10:54:14
3,Coari,BR,-4.0850,-63.1414,72.95,97,95,2.42,2021-02-10 10:59:07
4,Florø,NO,61.5996,5.0328,28.40,64,20,1.14,2021-02-10 10:59:08
...,...,...,...,...,...,...,...,...,...
596,Rexburg,US,43.8260,-111.7897,1.40,85,1,2.17,2021-02-10 11:01:06
597,Teya,RU,60.3778,92.6267,-4.50,87,100,5.95,2021-02-10 11:01:06
598,Eyl,SO,7.9803,49.8164,83.03,51,10,25.34,2021-02-10 10:59:15
599,Benguela,AO,-12.5763,13.4055,83.62,56,100,10.45,2021-02-10 11:01:07


In [8]:
# Narrow down to cities with (1) max temp less than 80*F, (2) max temp greater than 70*F, 
# (3) wind speed less than 10 MPH, (4) 0% cloudiness, and (5) humidity less than 80%
ideal_conditions = no_null_cities.loc[(no_null_cities['Max Temperature (*F)'] < 80) & 
                                      (no_null_cities['Max Temperature (*F)'] > 70) & 
                                      (no_null_cities['Wind Speed (MPH)'] < 10) & 
                                      (no_null_cities['Cloudiness (%)'] == 0) & 
                                      (no_null_cities['Humidity (%)'] < 80)]

narrow_cities_df = ideal_conditions.reset_index(drop=True)
narrow_cities_df.count()

City Name               9
Country                 9
Latitude                9
Longitude               9
Max Temperature (*F)    9
Humidity (%)            9
Cloudiness (%)          9
Wind Speed (MPH)        9
Datetime                9
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create lists to hold hotel API response data
h_names = []
h_cities = []
h_countries = []
h_lats = []
h_lngs = []

In [10]:
# Base url for API queries
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

count = 1

# Iterate through each row in hotel_df
for index, row in narrow_cities_df.iterrows():
    
    # Iteration latitude and longitude values
    i_lat = narrow_cities_df.iloc[index, 2]
    i_lng = narrow_cities_df.iloc[index, 3]
    
    # Save variables to use in params
    i_location = f'{i_lat}, {i_lng}'
    i_radius = 5000
    i_type = 'lodging'
    i_keyword = 'hotel'
    
    # Gmaps parameters dictionary
    params = {
        'location': i_location,
        'radius': i_radius,
        'types': i_type,
        'keyword': i_keyword,
        'key': g_key
    }
    
    # Call API request with response in .json format
    hotel_response = requests.get(base_url, params).json()
    
    # TEST ONLY
    # print(hotel_response.url)
    # hotel_response = hotel_response.json()
    
    # Print count and city
    print(f'Search {count} of {len(narrow_cities_df)}: Hotels near "{narrow_cities_df.iloc[index, 0]}" ')

    # Incorporate try/except to skip missing responses and continue iteration
    try:
        print(f"First Result: {hotel_response['results'][0]['name']}")
        h_names.append(hotel_response['results'][0]['name'])
        h_cities.append(narrow_cities_df.iloc[index, 0])
        h_countries.append(narrow_cities_df.iloc[index, 1])
        h_lats.append(hotel_response['results'][0]['geometry']['location']['lat'])
        h_lngs.append(hotel_response['results'][0]['geometry']['location']['lng'])
        
    except (IndexError):
        print(f'Sorry! No hotels matched search parameters for this city.')
        pass
        
    print(f'----------------------------------------------------------')
    
    count = count + 1

Search 1 of 9: Hotels near "Riyadh" 
First Result: Four Seasons Hotel Riyadh At Kingdom Center
----------------------------------------------------------
Search 2 of 9: Hotels near "Mīrābād" 
Sorry! No hotels matched search parameters for this city.
----------------------------------------------------------
Search 3 of 9: Hotels near "Marsá Maţrūḩ" 
First Result: Beau Site
----------------------------------------------------------
Search 4 of 9: Hotels near "Sirte" 
First Result: Salam Hotel
----------------------------------------------------------
Search 5 of 9: Hotels near "Ţahţā" 
First Result: El Salam Hotel
----------------------------------------------------------
Search 6 of 9: Hotels near "Najrān" 
Sorry! No hotels matched search parameters for this city.
----------------------------------------------------------
Search 7 of 9: Hotels near "Taoudenni" 
Sorry! No hotels matched search parameters for this city.
----------------------------------------------------------
Search 8 

In [11]:
# Tests
# print(h_names)
# print(h_cities)
# print(h_countries)
# print(h_lats)
# print(h_lngs)

In [12]:
hotel_dict = {
    'City': h_cities,
    'Country': h_countries,
    'Latitude': h_lats,
    'Longitude': h_lngs,
    'Hotel Name': h_names
}

hotel_df = pd.DataFrame(hotel_dict)
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Riyadh,SA,24.711491,46.674523,Four Seasons Hotel Riyadh At Kingdom Center
1,Marsá Maţrūḩ,EG,31.362287,27.220995,Beau Site
2,Sirte,LY,31.203933,16.574566,Salam Hotel
3,Ţahţā,EG,26.768816,31.504883,El Salam Hotel
4,Safaga,EG,26.763604,33.945966,Lamar Resort Abu Soma
5,Jalu,LY,29.032724,21.545805,Jalu Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))